In [ ]:
# main imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin # for creating custom transformers

Models classes:

In [ ]:
# models selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

Performance measure is **Root-Mean-Squared-Error (RMSE) between the logarithm** of the predicted value and the logarithm of the observed sales price.  
Taking logs means that errors in predicting expensive houses and cheap houses will affect the result almost equally

In [ ]:
from sklearn.metrics import mean_squared_log_error

Data preparation pipeline

In [ ]:
garage_section_feats = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond']
basement_section_feats = [
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'
]
columns_order = list(df_mod.columns)

garage_section_idxs = [columns_order.index(col) for col in garage_section_feats]
basement_section_idxs = [columns_order.index(col) for col in basement_section_feats]

print('Garage section indexes:\n', garage_section_idxs)
print('='*10)
print('Basement section indexes:\n', basement_section_idxs)

Custom transformers

In [ ]:
class RemoveBsmtGarageSections(BaseEstimator, TransformerMixin):
    def __init__(self, remove_bsmt=True, remove_garage=True) -> None:
        super().__init__()
        self.remove_garage = remove_garage
        self.remove_bsmt = remove_bsmt

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X


class FillNansAlley(BaseEstimator, TransformerMixin):
    def __init__(self, columns_and_values) -> None:
        #super().__init__()
        self.values_dict = columns_and_values

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X

In [ ]:
preparation_pipeline = Pipeline([
    ('nans_alley', FillNansAlley),
    ('std_scaler', StandardScaler())
])

In [ ]:
df_orig = pd.read_csv('datasets/processed/all_features_to_num_with_some_nans.csv')

In [ ]:
df_orig.info()

From now on 'df' will be the name of dataset we are working with. All the versions we save as backups or milestones will have the suffix starting with '_'. This suffix with the cell comment will help us to recover why we saved this version

In [1]:
df = df_orig.copy()

NameError: name 'df_orig' is not defined

What if we get rid of garage and basement sections but leave 'HasBsmt' and 'HasGarage' features?

In [ ]:
basement_features = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']
garage_features = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond']

In [ ]:
df = df_orig

In [ ]:
df_np_mod_experimental = df_np_mod.copy()
df_np_mod_experimental = df_np_mod_experimental.drop(columns=basement_features)
df_np_mod_experimental = df_np_mod_experimental.drop(columns=garage_features)
df_np_mod_experimental.info()